In [9]:
import pandas as pd
import math
import time
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize  

In [10]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\julie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\julie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
data=pd.read_csv("data/train.csv")
nlp = spacy.load('en_vectors_web_lg')

In [20]:
#extract week day and hour from the timestamp

def preprocess_day(timestamp):
    dico_day={"Mon":0,"Tue":1,"Wed":2,"Thu":3,"Fri":4,"Sat":5,"Sun":6}
    res=7*[0]
    res[dico_day[time.ctime(int(str(timestamp)[:-3]))[:3]]]=1
    return res

def preprocess_time(timestamp):
    res=24*[0]
    res[int(time.ctime(int(str(timestamp)[:-3]))[11:13])]=1
    return res

df=pd.DataFrame(data["timestamp"])
data["day"] = df.applymap(lambda x: preprocess_day(x))
data["time"] = df.applymap(lambda x: preprocess_time(x))
data = data.drop(columns="timestamp")

In [21]:
#replace the boolean by its int value
df=pd.DataFrame(data["user_verified"])
data["user_verified"]=df.applymap(lambda x: int(x))

In [22]:
#centrer & réduire
cols=["user_statuses_count","user_followers_count","user_friends_count"]
for c in cols:
    df=pd.DataFrame(data[c])
    m=data[c].mean()
    v=math.sqrt(data[c].var())
    data[c]=df.applymap(lambda x: (x-m)/v)

In [23]:
#count the number of 
def count_occ(s):
    if type(s)==str:
        return len(s.split(","))
    else:
        return 0

cols=["user_mentions","urls"]
for c in cols:
    df=pd.DataFrame(data[c])
    data[c]=df.applymap(lambda x: count_occ(x))

In [24]:
# text : keep only good words ie delete stop words
stop_words = set(stopwords.words('english')) 
def delete_stop(x) :
    # coronavirus is recognised by the nlp
    x = x.lower().replace("covid","coronavirus").replace("covid-19","coronavirus").replace("corona","coronavirus").replace("covid19","coronavirus")
    word_tokens = word_tokenize(x)  
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    return filtered_sentence

df=pd.DataFrame(data["text"])
data["clean_text"] = df.applymap(lambda x: delete_stop(x))

In [26]:
# the clean words will now be transformed into (300,) vectors
def text_to_vectors(x) :
    res = []
    for k in x :
        vec = nlp(k).vector
        if vec[0] != 0 :
            res.append(vec)
    return res

df=pd.DataFrame(data["clean_text"])
data["text_vect"] = df.applymap(lambda x: text_to_vectors(x))
data = data.drop(columns="text")

In [55]:
# hashtags
def to_low(x):
    try :
        res = math.isnan(x)
    except :
        return x.lower()

data["hashtags_low"] =pd.DataFrame(data["hashtags"]).applymap(lambda x: to_low(x))

def construc_dic() :
    dic = {}
    indice = 0
    for k in range(data.shape[0]) :
        hashtag = data["hashtags_low"][k]
        if hashtag != None :
            liste = hashtag.split(',')
            for word in liste :
                mot = word.replace(" ","")
                if not(mot in dic) :
                    dic[mot] = indice
                    indice +=1
    return dic

dic = construc_dic()

def hash_to_vec(x) :
    res = []
    if x != None :
        x = x.lower().split(',')
        for k in x :
            k = k.replace(" ","")
            res.append(dic[k])
    return res

data["hashtags_vec"] =pd.DataFrame(data["hashtags_low"]).applymap(lambda x: hash_to_vec(x))
        

In [56]:
data

,id,retweet_count,user_verified,user_statuses_count,user_followers_count,user_friends_count,user_mentions,urls,hashtags,day,time,clean_text,text_vect,hashtags_low,hashtags_vec
0,0,0,0,0.271991,-0.094948,-0.087929,0,0,NaN,"[0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Smh, I, give]","[[-0.38512, -0.1739, 0.25749, -0.48121, -0.136...",None,[]
1,1,0,0,-0.420002,-0.095378,-0.147277,0,0,NaN,"[0, 0, 0, 0, 0, 0, 1]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Most, us, Human, Beings, ,, I, think, miss, b...","[[-0.19571, 0.056275, -0.096518, -0.26408, 0.3...",None,[]
2,2,0,0,-0.390231,-0.094713,-0.024234,0,0,NaN,"[0, 1, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Old, dirty, tricks, Trump, ,, ..., like, n't,...","[[0.26105, -0.043804, -0.3964, 0.022796, -0.04...",None,[]
3,3,0,0,-0.094897,-0.095125,-0.141365,0,0,NaN,"[0, 0, 0, 0, 0, 1, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Seriously, ....., I, worked, 86, hours, last,...","[[-0.28038, 0.020129, -0.19906, -0.14018, -0.2...",None,[]
4,4,0,0,-0.417240,-0.095381,-0.151624,0,0,NaN,"[1, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[May, ALMIGHTY, ALLAH, mercy, us, ., Only, lag...","[[-0.042501, 0.090773, -0.11918, 0.12372, -0.1...",None,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665772,665772,0,0,0.240463,-0.094586,-0.048750,0,0,NaN,"[0, 0, 0, 0, 0, 1, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[18, months, dawg, ?, Come, man, ...]","[[-0.10543, 0.83512, 0.60768, 0.079782, -0.002...",None,[]
665773,665773,1,0,-0.404791,-0.094568,-0.138873,1,1,COVID19,"[0, 0, 0, 0, 1, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[Thank, nurses, @, StanfordEMED, family, shari...","[[-0.32633, 0.2267, -0.6039, -0.079733, -0.313...",covid19,[3]
665774,665774,8,0,-0.414123,-0.095364,-0.144089,0,1,NaN,"[0, 0, 0, 0, 1, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","['Post, ', pearls, Palliative, ,, End, Life, B...","[[-0.29729, 0.583285, -0.066182494, 0.15645, 0...",None,[]
665775,665775,0,0,-0.418550,-0.095383,-0.153710,0,0,NaN,"[0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[His, facial, expressions, kind, looking, answ...","[[0.019097, 0.23119, -0.1681, 0.1299, 0.34298,...",None,[]


In [57]:
data.to_csv("data/train_clean.csv")

KeyboardInterrupt: 